# Synthetic seismic: wedge

We're going to make the famous wedge model, which interpreters can use to visualize the tuning effect. Then we can extend the idea to other kinds of model.

## Make a wedge earth model

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
length = 80  # x range
depth = 200  # z range

### EXERCISE

Make a NumPy array of integers with these dimensions, placing a boundary at a 'depth' of 66 and another at a depth of 133.

A plot of a vertical section through this array should look something like:

      |
      |
      ---
        |
        |
        ---
          |
          |

In [ ]:
# YOUR CODE HERE



In [ ]:
# We have to pass dtype=int or we get floats.
# We need ints because we're going to use for indexing later.
model = 1 + np.tri(depth, length, -depth//3, dtype=int)

plt.imshow(model)
plt.colorbar()
plt.show()

Now set the upper part of the model — above the wedge — to zero.

In [ ]:
model[:depth//3,:] = 0

plt.imshow(model)
plt.colorbar()
plt.show()

Now we can make some Vp-rho pairs (rock 0, rock 1, and rock 2).

In [ ]:
rocks = np.array([[2540, 2550],   # <-- Upper layer
                  [2400, 2450],   # <-- Wedge
                  [2650, 2800]])  # <-- Lower layer

Now we can use ['fancy indexing'](http://docs.scipy.org/doc/numpy/user/basics.indexing.html) to use `model`, which is an array of 0, 1, and 2, as the indices of the rock property pairs to 'grab' from `rocks`.

In [ ]:
earth = rocks[model]

Now apply `np.prod` (product) to those Vp-rho pairs to get impedance at every sample.

In [ ]:
imp = np.apply_along_axis(np.prod, arr=earth, axis=-1)

## Model seismic reflections

Now we have an earth model — giving us acoustic impedance everywhere in this 2D grid — we define a function to compute reflection coefficients for every trace.

### EXERCISE

Can you write a function to compute the reflection coefficients in this model?

It should implement this equation, where $Z$ is acoustic impedance and :

$$ R = \frac{Z_\mathrm{lower} - Z_\mathrm{upper}}{Z_\mathrm{lower} + Z_\mathrm{upper}} $$

The result should be a sparse 2D array of shape (199, 80). The upper interface of the wedge should be positive.

In [ ]:
def make_rc(imp):
    
    # YOUR CODE HERE
    
    return rc

rc = make_rc(imp)

In [ ]:
def make_rc(imp):
    """
    Compute reflection coefficients.
    """
    upper = imp[ :-1, :]
    lower = imp[1:  , :]
    
    return (lower - upper) / (lower + upper)

rc = make_rc(imp)

You should be able to plot the RC series like so:

In [ ]:
plt.figure(figsize=(8,4))
plt.imshow(rc, aspect='auto')
plt.colorbar()
plt.show()

### EXERCISE

Implement a Ricker wavelet of frequency $f$ with amplitude $A$ at time $t$ given by:

$$ \mathbf{a}(\mathbf{t}) = (1-2 \pi^2 f^2 \mathbf{t}^2) \mathrm{e}^{-\pi^2 f^2 \mathbf{t}^2} $$

In [ ]:
# YOUR CODE HERE



There is an implementation in `scipy.signal` but it has a 'width parameter' instead of 'frequency' so it's harder to parameterize.

Instead, we'll use `bruges` to make a wavelet:

In [ ]:
from bruges.filters import ricker

f = 25  # We'll use this later.
w, t = ricker(duration=0.128, dt=0.001, f=f, return_t=True)

plt.plot(t, w)
plt.show()

### EXERCISE

Make an RC series 200 samples long, with one positive and one negative RC. Make a corresponding time array.

Pass the RC series to `np.convolve()` along with the wavelet, then plot the resulting synthetic seismogram.

In [ ]:
# YOUR CODE HERE



In [ ]:
temp = np.zeros(200)
temp[66] = 1
temp[133] = -0.5

tr = np.convolve(temp, w, mode='same')

plt.plot(tr)

## Synthetic wedge

It's only a little trickier for us to apply 1D convolution to every trace in our 2D reflection coeeficient matrix. NumPy provides a function, `apply_along_axis()` to apply any function along any one axis of an n-dimensional array. I don't think it's much faster than looping, but I find it easier to think about.

In [ ]:
def convolve(trace, wavelet):
    return np.convolve(trace, wavelet, mode='same')

synth = np.apply_along_axis(convolve,
                            axis=0,
                            arr=rc,
                            wavelet=w)

plt.figure(figsize=(12,6))
plt.imshow(synth, cmap="Greys", aspect=0.2)
plt.colorbar()
plt.show()

### EXERCISE

Use `ipywidgets.interact` to turn this into an interactive plot, so that we can vary the frequency of the wavelet and see the effect on the synthetic.

Here's a reminder of how to use it:

    from ipywidgets import interact

    @interact(a=(0, 10, 1), b=(0, 100, 10))
    def main(a, b):
        """Do the things!"""
        print(a + b)
        return

In [ ]:
# YOUR CODE HERE



In [ ]:
from ipywidgets import interact

@interact(f=(4, 100, 4))
def show(f):
    w, t = ricker(duration=0.128, dt=0.001, f=f, return_t=True)
    synth = np.apply_along_axis(convolve,
                                axis=0,
                                arr=rc,
                                wavelet=w)
    plt.figure(figsize=(12,6))
    plt.imshow(synth, cmap="Greys", aspect=0.2)
    plt.colorbar()
    plt.show()

<hr />

<div>
<img src="https://avatars1.githubusercontent.com/u/1692321?s=50"><p style="text-align:center">© Agile Scientific 2020</p>
</div>